# Structure gemini outputs in a robust way

In [1]:
from google import genai


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """
)
print(response.text)

Oh, get ready for a culinary journey! Here are a few irresistible Asian soup recipes, designed to warm your soul and tantalize your taste buds:

---

### 1. Vietnamese Pho Bo (Beef Noodle Soup)

**A Yummy Description:** Imagine a bowl of pure comfort, brimming with an incredibly aromatic and deeply savory beef broth, slow-simmered for hours with fragrant star anise, cinnamon, ginger, and cardamom. This soul-warming elixir cradles slippery, tender rice noodles, thinly sliced rare beef that cooks gently in the hot broth, and is crowned with a vibrant explosion of fresh herbs like cilantro and basil, crunchy bean sprouts, and a squeeze of lime. It's an elegant dance of subtle flavors, utterly irresistible and profoundly satisfying.

**Ingredients:**

*   **For the Broth:**
    *   2-3 lbs beef bones (knuckles, marrow bones, oxtail)
    *   1-2 lbs beef chuck, brisket, or flank (for the soup meat)
    *   1 large onion, halved and charred
    *   1 large piece ginger, sliced and charred
  

In [2]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients. Structure it in json format
    """
)
print(response.text)

```json
[
  {
    "name": "Tom Yum Goong (Thai Spicy Shrimp Soup)",
    "description": "A vibrant and zesty Thai hot and sour soup, bursting with aromatic herbs like lemongrass and galangal, plump shrimp, and a fiery kick from chilies that will awaken your taste buds. It's a symphony of sour, spicy, salty, and sweet!",
    "ingredients": [
      "Fresh shrimp (peeled and deveined)",
      "Lemongrass stalks (bruised and sliced)",
      "Galangal (sliced)",
      "Kaffir lime leaves (torn)",
      "Mushrooms (straw or oyster)",
      "Fish sauce",
      "Fresh lime juice",
      "Thai bird's eye chilies (crushed or sliced)",
      "Chili paste (Nam Prik Pao, optional for richer color)",
      "Cilantro (fresh, for garnish)",
      "Chicken or shrimp broth"
    ]
  },
  {
    "name": "Pho Bo (Vietnamese Beef Noodle Soup)",
    "description": "Vietnam's beloved, deeply fragrant noodle soup features a mesmerizing beef broth simmered for hours with star anise, cinnamon, and ginger. It’s ser

In [1]:
from pydantic import BaseModel
from google import genai


class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]},
)

print(response.text)

[{"recipe_name":"Ramen (Shoyu)","description":"A comforting Japanese noodle soup with a rich soy-sauce based broth, tender chashu pork, a marinated soft-boiled egg, and bouncy noodles.","ingredients":["Ramen noodles","Chashu pork","Soy sauce","Mirin","Sake","Chicken or pork broth","Narutomaki (fish cake)","Scallions","Nori (seaweed)","Soft-boiled egg"]},
{"recipe_name":"Tom Yum Goong","description":"A classic Thai hot and sour shrimp soup, bursting with aromatic herbs like lemongrass, galangal, and kaffir lime leaves, and a spicy kick from chilies.","ingredients":["Shrimp","Lemongrass","Galangal","Kaffir lime leaves","Mushrooms","Fish sauce","Lime juice","Chili paste (nahm prik pao)","Fresh chilies","Cilantro"]},
{"recipe_name":"Pho Bo (Vietnamese Beef Noodle Soup)","description":"A fragrant Vietnamese beef noodle soup with a deeply savory broth, delicate rice noodles, tender slices of beef, and a medley of fresh herbs and sprouts.","ingredients":["Beef bones","Beef brisket or sirloin"

In [5]:
recipes = response.parsed
type(recipes), len(recipes)

(list, 4)

In [6]:
recipes[0]

Recipe(recipe_name='Ramen (Shoyu)', description='A comforting Japanese noodle soup with a rich soy-sauce based broth, tender chashu pork, a marinated soft-boiled egg, and bouncy noodles.', ingredients=['Ramen noodles', 'Chashu pork', 'Soy sauce', 'Mirin', 'Sake', 'Chicken or pork broth', 'Narutomaki (fish cake)', 'Scallions', 'Nori (seaweed)', 'Soft-boiled egg'])

In [9]:
recipes[-1].recipe_name, recipes[-1].ingredients

('Miso Soup',
 ['Dashi stock', 'Miso paste', 'Silken tofu', 'Wakame seaweed', 'Scallions'])

## Simulate house prices

In [10]:
from typing import Literal

class Home(BaseModel):
    price: int 
    monthly_fee: int 
    living_area: float 
    number_rooms: float
    address: str
    type: Literal["apartment", "house"]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""List 20 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK.""",
    config={"response_mime_type": "application/json", "response_schema": list[Home]},
)

homes = response.parsed
homes[:3]

[Home(price=3500000, monthly_fee=2500, living_area=75.5, number_rooms=3.0, address='Västra Hamngatan 12, Göteborg', type='apartment'),
 Home(price=6800000, monthly_fee=4000, living_area=140.2, number_rooms=5.0, address='Storgatan 50, Malmö', type='house'),
 Home(price=2850000, monthly_fee=2800, living_area=60.0, number_rooms=2.0, address='Drottninggatan 8, Uppsala', type='apartment')]

In [15]:
import pandas as pd 

df = pd.DataFrame([dict(home) for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,address,type
0,3500000,2500,75.5,3.0,"Västra Hamngatan 12, Göteborg",apartment
1,6800000,4000,140.2,5.0,"Storgatan 50, Malmö",house
2,2850000,2800,60.0,2.0,"Drottninggatan 8, Uppsala",apartment
3,4200000,3500,105.8,4.0,"Kungsbackavägen 25, Kungsbacka",house
4,5100000,2200,88.0,3.5,"Karlavägen 75, Stockholm",apartment


In [17]:
df.query("price < 2_000_000")

,price,monthly_fee,living_area,number_rooms,address,type
12,1900000,2600,45.0,1.0,"Kungsgatan 3, Örebro",apartment
